In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
data = pd.read_csv('Data/train_rosbank.csv')
data['TRDATETIME'] = pd.to_datetime(data['TRDATETIME'], format='%d%b%y:%X')
data.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,2017-10-21 00:00:00,5023.000,POS,0,0.000
1,01/10/2017,0,6011,NaN,810,2017-10-12 12:24:07,20000.000,DEPOSIT,0,0.000
2,01/12/2017,0,5921,NaN,810,2017-12-05 00:00:00,767.000,POS,0,0.000
3,01/10/2017,0,5411,NaN,810,2017-10-21 00:00:00,2031.000,POS,0,0.000
4,01/10/2017,0,6012,NaN,810,2017-10-24 13:14:24,36562.000,C2C_OUT,0,0.000


In [3]:
data['channel_type'] = data['channel_type'].fillna('0')

In [4]:
# Пересчитаем валютные транзакции в рубли по курсам цб
mapping = {810: 1, 978: 65.87, 504: 6.5, 704: 0.0025, 981: 23, 985: 15.48,
           840: 58.31, 949: 15.97, 51: 4.3, 826: 75.11, 214: 1.23, 764: 1.7,
           203: 2.5, 702: 42, 360: 0.0042, 756: 59.26, 933: 30.20, 975: 33.68,
           36: 13.5, 191: 8.5, 784: 15.9, 980: 2.2, 124: 44.97, 398: 0.18,
           376: 15.7, 944: 33.81, 352: 0.52, 417: 0.85, 156: 8.6, 752: 6.8,
           392: 0.52, 484: 3.3, 634: 15.8, 188: 0.11, 643: 1, 348: 0.21, 356: 0.90,
           458: 13.5, 986: 18, 498: 3.5, 578: 7.1, 208: 8.9, 344: 7.5, 32: 65.87,
           410: 0.05, 788: 24, 480: 1.6, 604: 17.5, 941: 0.55, 144: 0.37, 946: 14.4,
           710: 4.52, 690: 4, 44: 20, 170: 0.02, 901: 1.9, 608: 1.15, 554: 41.5, 462: 3.7}
data['amount'] = data['amount'] * data['currency'].map(mapping)

In [5]:
%%time

def build_features(data):
    aggregated = data.groupby('cl_id')[['channel_type']].first()
    ids = aggregated.index
    aggregated['cl_id'] = ids
    aggregated['total_number_transact_cl_id'] = data.groupby(
        'cl_id')['TRDATETIME'].apply(lambda x: x.count())
    aggregated['uniq_number_MCC_cl_id'] = data.groupby(
        'cl_id')['MCC'].apply(lambda x: x.nunique())
    data['uniq_number_MCC_cl_id_period'] = data.groupby(
        ['cl_id', 'PERIOD'])['MCC'].transform(lambda x: x.nunique())
    aggregated['uniq_number_MCC_cl_id_period'] = data.groupby(
        'cl_id')['uniq_number_MCC_cl_id_period'].apply(lambda x: x.mean())
    aggregated['trx_category_cl_id_mode'] = data.groupby(
        'cl_id')['trx_category'].apply(lambda x: x.value_counts().index[0])
    aggregated['diff_days_cl_id'] = data.groupby(
        'cl_id')['TRDATETIME'].apply(lambda x: x.max() - x.min()).dt.days
    data['diff_days_cl_id_period'] = data.groupby(
        ['cl_id', 'PERIOD'])['TRDATETIME'].transform(
        lambda x: x.max() - x.min()).dt.days
    aggregated['diff_days_cl_id_period'] = data.groupby(
        'cl_id')['diff_days_cl_id_period'].apply(lambda x: x.mean())
    data['days_from_first_transaction'] = data.groupby(
        'cl_id')['TRDATETIME'].transform(lambda x: x - x.min()).dt.days
    aggregated['days_from_first_transaction'] = data.groupby(
        'cl_id')['days_from_first_transaction'].apply(lambda x: x.mean())
    aggregated['mad_of_days_from_first_transaction'] = data.groupby(
        'cl_id')['days_from_first_transaction'].apply(lambda x: x.mad())
    data['trx_type'] = data.apply(
        lambda x: 1 if x['trx_category'] in ['DEPOSIT', 'C2C_IN', 'BACK_TRX'] else -1, axis=1)
    data['amount_signed'] = data['amount'] * data['trx_type']
    aggregated['amount_signed'] = data.groupby(
        'cl_id')['amount_signed'].apply(lambda x: x.mean()) 
    aggregated['amount_sum'] = data.groupby(
        'cl_id')['amount'].apply(lambda x: x.sum())
    aggregated['amount_signed_sum'] = data.groupby(
        'cl_id')['amount_signed'].apply(lambda x: x.sum())
    aggregated['amount_snyatie'] = (aggregated['amount_sum'] - aggregated['amount_signed_sum']) / 2
    aggregated['amount_popolnenie'] = aggregated['amount_sum'] - aggregated['amount_snyatie']
    aggregated['target_flag'] = data.groupby('cl_id')['target_flag'].first()           
    return aggregated
data_agg2 = build_features(data)

CPU times: user 35.7 s, sys: 673 ms, total: 36.4 s
Wall time: 28.5 s


In [6]:
data_agg2.head(10)

,channel_type,cl_id,total_number_transact_cl_id,uniq_number_MCC_cl_id,uniq_number_MCC_cl_id_period,trx_category_cl_id_mode,diff_days_cl_id,diff_days_cl_id_period,days_from_first_transaction,mad_of_days_from_first_transaction,amount_signed,amount_sum,amount_signed_sum,amount_snyatie,amount_popolnenie,target_flag
cl_id,,,,,,,,,,,,,,,,
0,0,0,5,5,3.400,POS,53,9.600,16.200,14.720,-4876.600,64383.000,-24383.000,44383.000,20000.000,0
1,0,1,104,23,14.423,POS,92,24.596,59.298,17.724,-1297.905,324982.152,-134982.152,229982.152,95000.000,0
5,0,5,142,34,16.338,POS,92,23.697,47.951,23.597,-1899.197,557666.027,-269686.027,413676.027,143990.000,1
9,0,9,39,5,3.410,POS,89,26.128,43.667,25.744,-6292.695,849315.090,-245415.090,547365.090,301950.000,0
10,0,10,463,36,23.469,POS,89,28.758,44.423,19.295,-543.724,1124343.990,-251743.990,688043.990,436300.000,0
11,0,11,217,39,20.724,POS,91,26.332,39.203,20.694,-318.963,433215.032,-69215.032,251215.032,182000.000,0
14,0,14,136,30,14.574,POS,92,20.706,57.529,22.541,-1053.604,448690.170,-143290.170,295990.170,152700.000,1
20,0,20,77,17,12.156,POS,73,25.000,43.948,19.588,-94.910,437308.090,-7308.090,222308.090,215000.000,0
21,0,21,124,35,18.048,POS,88,27.387,44.363,21.257,-817.142,338881.650,-101325.650,220103.650,118778.000,0


In [7]:
data_agg2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 10215
Data columns (total 16 columns):
channel_type                          5000 non-null object
cl_id                                 5000 non-null int64
total_number_transact_cl_id           5000 non-null int64
uniq_number_MCC_cl_id                 5000 non-null int64
uniq_number_MCC_cl_id_period          5000 non-null float64
trx_category_cl_id_mode               5000 non-null object
diff_days_cl_id                       5000 non-null int64
diff_days_cl_id_period                5000 non-null float64
days_from_first_transaction           5000 non-null float64
mad_of_days_from_first_transaction    5000 non-null float64
amount_signed                         5000 non-null float64
amount_sum                            5000 non-null float64
amount_signed_sum                     5000 non-null float64
amount_snyatie                        5000 non-null float64
amount_popolnenie                     5000 non-null float64
ta

In [8]:
y = data_agg2.pop('target_flag')
data_agg2.drop('channel_type', axis=1, inplace=True)
X = pd.get_dummies(data_agg2)

In [9]:
import lightgbm as lgb
kf = KFold(n_splits=5, shuffle=True, random_state=42)
lgboost = lgb.LGBMClassifier(n_estimators=500, max_depth=1, learning_rate=0.1,
                             random_state=42)
cross_val_score(lgboost, X, y, cv=kf, scoring='roc_auc').mean()

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


0.8501735333341586